In [1]:
import pymysql
import pyforest

In [2]:
def acess_db():
    '''
        Function to connect the database.
    '''
    db = pymysql.connect(
            host="213.190.6.47",    # your host, usually localhost
            user="u698076480_admin",         # your username
            passwd="hGa8yG7Q",  # your password
            db="u698076480_datalab"
        )
    return db

In [3]:
def columns_db(table):
    '''
        Function to return a list with the columns present in a specific table.

        Parameters:
            table: Name (str) of the table to be checked.

        Returns:
            table: List with column names.
    '''
    sql_query = f"SHOW COLUMNS FROM `{table}`"
    db = acess_db()
    cur = db.cursor()
    cur.execute(sql_query)
    tabela = [[v for k, v in enumerate(row)][0] for row in cur.fetchall()]
    db.close()
    return tabela

In [4]:
def recover_db(sql_query, table):
    '''
        Function to access information in a table.
        
        Parameters:
            table: Table (str) to be checked.
            sql_query: SQL script (str) to filter the table information.
            
        Return:
            table: DataFrame pandas containing the requested information.
    '''
    db = acess_db()
    cur = db.cursor()
    cur.execute(sql_query)
    tabela = {row[0]: [v for k, v in enumerate(row)] for row in cur.fetchall()}
    db.close()
    df = pd.DataFrame(tabela).transpose()
    df.columns = columns_db(table)
    df.set_index(columns_db(table)[0], inplace=True)
    return df

In [5]:
def sql_simples(sql_query):
    '''
        Simplified SQL execution function.
        
        Parameters:
            mysql_query: SQL script (str) to execute.
                        DROP, CREATE, ALTER, DELETE can be used.
    '''
    db = acess_db()
    cur = db.cursor()
    cur.execute(sql_query)
    db.commit()
    db.close()

In [6]:
def sql_mult(sql, val):
    '''
        Simplified SQL execution function.
        
        Parameters:
            mysql_query: SQL script (str) to execute.
                        INSERT, UPDATE can be used.
    '''
    db = acess_db()
    cur = db.cursor()
    cur.executemany(sql, val)
    db.commit()
    db.close()

In [7]:
tabela_trabalho = 'design'
df_mysql = recover_db(f'SELECT * FROM `{tabela_trabalho}`', tabela_trabalho)
df_mysql.tail()

<IPython.core.display.Javascript object>

,date,ip,tcle,orig,fig,per1,per2,per3,per4,per5,per6,per7,per8,per9,per10,per11
id,,,,,,,,,,,,,,,,
468,1604069950,216.48.86.237,s,en,66,5,5,1,5,4,5,2,5,4,4,3
469,1604076795,129.85.89.83,s,en,1,2,2,1,2,2,1,1,2,2,4,3
470,1604336897,201.241.0.39,s,en,11,3,5,1,2,5,4,1,2,4,4,3
471,1607367003,189.26.122.99,s,en,2,5,4,2,4,5,4,1,5,5,4,3
472,1607449558,2804:14c:87c3:81d3:8c17:8917:c6fd:b264,s,en,22,5,4,1,4,5,4,1,3,3,4,3


In [8]:
df_mysql['date'] = pd.to_datetime(df_mysql['date'], unit='s')
# df_mysql.tail()

<IPython.core.display.Javascript object>

In [9]:
num = ['per1', 'per2', 'per4', 'per5', 'per6', 'per8', 'per9']
df_mysql[num] = df_mysql[num].astype(int)
df_mysql.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 472 entries, 1 to 472
Data columns (total 16 columns):
date     472 non-null datetime64[ns]
ip       472 non-null object
tcle     472 non-null object
orig     472 non-null object
fig      472 non-null object
per1     472 non-null int32
per2     472 non-null int32
per3     472 non-null object
per4     472 non-null int32
per5     472 non-null int32
per6     472 non-null int32
per7     472 non-null object
per8     472 non-null int32
per9     472 non-null int32
per10    472 non-null object
per11    472 non-null object
dtypes: datetime64[ns](1), int32(7), object(8)
memory usage: 49.8+ KB


In [10]:
df_mysql['Origem'] = (df_mysql['fig'] > 7)
original = {
    True: 'Designer',
    False: 'Pesquisador',
}
df_mysql['Origem'].replace(original, inplace=True)
df_mysql.tail()

,date,ip,tcle,orig,fig,per1,per2,per3,per4,per5,per6,per7,per8,per9,per10,per11,Origem
id,,,,,,,,,,,,,,,,,
468,2020-10-30 14:59:10,216.48.86.237,s,en,66,5,5,1,5,4,5,2,5,4,4,3,Designer
469,2020-10-30 16:53:15,129.85.89.83,s,en,1,2,2,1,2,2,1,1,2,2,4,3,Pesquisador
470,2020-11-02 17:08:17,201.241.0.39,s,en,11,3,5,1,2,5,4,1,2,4,4,3,Designer
471,2020-12-07 18:50:03,189.26.122.99,s,en,2,5,4,2,4,5,4,1,5,5,4,3,Pesquisador
472,2020-12-08 17:45:58,2804:14c:87c3:81d3:8c17:8917:c6fd:b264,s,en,22,5,4,1,4,5,4,1,3,3,4,3,Designer


In [11]:
fig_names = {
    1: 'Reovírus Aviário',
    2: 'Células Tronco',
    3: 'T. gondii',
    4: 'Citocinas',
    5: 'Febre Amarela',
    6: 'Linfócitos',
    11: 'Reovírus Aviário',
    22: 'Células Tronco',
    33: 'T. gondii',
    44: 'Citocinas',
    55: 'Febre Amarela',
    66: 'Linfócitos',
}
df_mysql['fig'].replace(fig_names, inplace=True)
# df_mysql.tail()

In [12]:
grau = {
    1: 'Graduação',
    2: 'Pós-Graduação',
    3: 'Mestrado',
    4: 'Doutorado',
}
df_mysql['per10'].replace(grau, inplace=True)
# df_mysql.tail()

In [13]:
forma = {
    1: 'Humanas',
    2: 'Exatas',
    3: 'Biológicas'
}
df_mysql['per11'].replace(forma, inplace=True)
# df_mysql.tail()

In [14]:
bina = {
    1: 'Sim',
    2: 'Não'
}
df_mysql['per3'].replace(bina, inplace=True)
df_mysql['per7'].replace(bina, inplace=True)
# df_mysql.tail()

In [15]:
colunas = [
    'Data',
    'IP',
    'TCLE',
    'Língua',
    'Figura',
    'Dintinguíveis',
    'Proeminentes',
    'Descrição',
    'Eficiente',
    'Útil',
    'Estética',
    'Fonte',
    'Compreensão',
    'Interesse',
    'Grau',
    'Área',
    'Origem',
]
df_mysql.columns = colunas
joblib.dump(df_mysql, 'dados\df_mysql.gz')
df_mysql.tail()

<IPython.core.display.Javascript object>

,Data,IP,TCLE,Língua,Figura,Dintinguíveis,Proeminentes,Descrição,Eficiente,Útil,Estética,Fonte,Compreensão,Interesse,Grau,Área,Origem
id,,,,,,,,,,,,,,,,,
468,2020-10-30 14:59:10,216.48.86.237,s,en,Linfócitos,5,5,Sim,5,4,5,Não,5,4,Doutorado,Biológicas,Designer
469,2020-10-30 16:53:15,129.85.89.83,s,en,Reovírus Aviário,2,2,Sim,2,2,1,Sim,2,2,Doutorado,Biológicas,Pesquisador
470,2020-11-02 17:08:17,201.241.0.39,s,en,Reovírus Aviário,3,5,Sim,2,5,4,Sim,2,4,Doutorado,Biológicas,Designer
471,2020-12-07 18:50:03,189.26.122.99,s,en,Células Tronco,5,4,Não,4,5,4,Sim,5,5,Doutorado,Biológicas,Pesquisador
472,2020-12-08 17:45:58,2804:14c:87c3:81d3:8c17:8917:c6fd:b264,s,en,Células Tronco,5,4,Sim,4,5,4,Sim,3,3,Doutorado,Biológicas,Designer


In [16]:
perguntas = {
    colunas[4 + 1]: '1) Na imagem, o quão claramente os elementos são distinguíveis?',
    colunas[4 + 2]: '2) O quão visualmente proeminentes são os elementos da imagem?',
    colunas[4 + 3]: '8) Existem elementos da imagem que requerem descrição adicional?',
    colunas[4 + 4]: '3) O quão eficiente são os elementos da imagem na representação dos dados?',
    colunas[4 + 5]: '4) O quão útil é a aplicação de uma imagem esquemática – desenho científico ou graphic abstract para representar os dados?',
    colunas[4 + 6]: '5) O quanto a imagem é esteticamente bem elaborada?',
    colunas[4 + 7]: '9) A imagem apresentada faz você experimentar uma nova fonte de dados?',
    colunas[4 + 8]: '6) Classifique o quanto a imagem facilitou a compreensão do conteúdo da informação.',
    colunas[4 + 9]: '7) Classifique o quanto a imagem despertou o seu interesse pelo conteúdo.',
    colunas[4 + 10]: 'Qual o seu grau de formação acadêmica?',
    colunas[4 + 11]: 'Área de formação?',
}
joblib.dump(perguntas, 'dados\perguntas.gz')
printDic(perguntas)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

------------------
-=< Dicionário >=-
------------------
Dintinguíveis -> 1) Na imagem, o quão claramente os elementos são distinguíveis?
Proeminentes -> 2) O quão visualmente proeminentes são os elementos da imagem?
Descrição -> 8) Existem elementos da imagem que requerem descrição adicional?
Eficiente -> 3) O quão eficiente são os elementos da imagem na representação dos dados?
Útil -> 4) O quão útil é a aplicação de uma imagem esquemática – desenho científico ou graphic abstract para representar os dados?
Estética -> 5) O quanto a imagem é esteticamente bem elaborada?
Fonte -> 9) A imagem apresentada faz você experimentar uma nova fonte de dados?
Compreensão -> 6) Classifique o quanto a imagem facilitou a compreensão do conteúdo da informação.
Interesse -> 7) Classifique o quanto a imagem despertou o seu interesse pelo conteúdo.
Grau -> Qual o seu grau de formação acadêmica?
Área -> Área de formação?
